In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
farming_2020 = gpd.read_file(data_folder + "clean_data/farming_census_2020/farming_census_2020.shp")
grazing_2020 = gpd.read_file(data_folder + "clean_data/grazing_census_2020/grazing_census_2020.shp")
land_2020 = gpd.read_file(data_folder + "clean_data/agricultural_land_usage_census_2020/agricultural_land_usage_census_2020.shp")
farming_2010 = gpd.read_file(data_folder + "clean_data/farming_land_usage_census_2010/farming_land_usage_census_2010.shp")
grazing_2010 = gpd.read_file(data_folder + "clean_data/grazing_census_2010/grazing_census_2010.shp")

---

# 2020

In [ ]:
df2020 = land_2020.merge(grazing_2020.merge(farming_2020, on='REF_AREA', how='inner'), on='REF_AREA', how='inner')
for c in df2020.columns:
    print(c)

In [ ]:
print((df2020.geometry != df2020.geometry_x).sum())
print((df2020.geometry_x != df2020.geometry_y).sum())

In [ ]:
df2020.drop(columns=['geometry_x', 'geometry_y'], inplace=True)

In [ ]:
# C = df2020[df2020.columns.drop(['REF_AREA', 'geometry', 'ALL'])].corr(method='spearman')

# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(C, ax=ax, cmap='coolwarm', vmin=-1)
# plt.show()

---

# 2010

In [ ]:
df2010 = farming_2010.merge(grazing_2010, on='Codice Com', how='inner')
(df2010.geometry_x != df2010.geometry_y).sum()

In [ ]:
df2010 = df2010.drop(columns='geometry_x').rename({'geometry_y': 'geometry'}, axis=1)

In [ ]:
df2010.info()

In [ ]:
df2010.drop(columns=df2010.columns[[0, 34, 33]], inplace=True)

---

# Uniformise variables

In [ ]:
grazing_dict={'TOTCATT': 'totale bovini', 
              'TOTBUFF': 'totale bufalini', 
              'TOTSH': 'totale ovini', 
              'TOTG': 'totale caprini', 
              'TOTPIG': 'totale suini',
              'TOTPOU': 'totale avicoli',
              'HIVES': 'totale alveari',
              'totale bov': 'totale bovini',
              'totale sui': 'totale suini',
              'totale ovi': 'totale ovini',
              'totale avi': 'totale avicoli'
}
# gather TOTCATT + TOTBUFF, TOTG + TOTSH, share HIVES

In [ ]:
df2020.loc[:, 'TOTCATT'] = df2020['TOTCATT'] + df2020['TOTBUFF']
df2020.loc[:, 'TOTSH'] = df2020['TOTG'] + df2020['TOTSH']
df2020.drop(columns=['TOTG', 'TOTBUFF'], inplace=True)
df2010 = df2010.merge(df2020[['HIVES', 'geometry']], on='geometry', how='left')
df2010.rename(grazing_dict, axis=1, inplace=True)
df2020.rename(grazing_dict, axis=1, inplace=True)

---

In [ ]:
farming_dict = {'ARLAND': ('seminativi', 'seminativi'),
    'CEREAG': ('cereali pe', 'cereali per la produzione di granella'), 
    'OTHDRYLEG': ('altri legumi secchi e colture proteiche'),
                'POTAT': ('patata', 'patata'),
                'BEETS': ('barbabieto', 'barbabietola da zucchero'),
                'FODR': ('piante sar', 'piante sarchiate da foraggio'),
                'CROPIN': ('piante ind', 'piante industriali'),
                'VEGCO_TUN': ('ortive', "ortive protette in serra e tunnel accessibili all'uomo"),
                'FLOWO': ('fiori e pi', 'fiori e piante ornamentali in piena aria'),
                'FLOWGH_TUN': ('fiori e pi', "fiori e piante ornamentali in serra e tunnel accessibili all'uomo"),
                'SEEDL_SEEDS': ('piantine', 'sementi e piantine'),
                'SEEDL_SEEDS': ('sementi', 'sementi e piantine'),
                'TGRA': ('foraggere', 'foraggere avvicendate'),
                'FALLAND': ('terreni a', 'terreni a riposo'),
                'PCROPS': ('coltivazio', 'coltivazioni legnose agrarie'),
                'VINEY': ('vite', 'vite'),
                'OLIVOOILTR': ('olivo per', 'olivo per la produzione di olive da olio'),
                'CITRFR': ('agrumi', 'agrumi'),
                'FRUIT': ('fruttiferi', 'coltivazioni fruttifere'),
                'NURSEOR': ('vivai', 'piante ornamentali da vivaio'),
                'PCROO': ('altre colt', 'altre coltivazioni legnose agrarie'),
                'PGRAPM': ('prati perm', 'prati permanenti e pascoli')
}